# CRIME IN LOS ANGELES
    Crime data from 2010 through September 2017

In [67]:
#TEAM MEMBERS:
#              Sylvester Zowonu
#              Frank Navarette
#              Max Ru
#Course      : CS 4661 (Data Science)
#Instructor  : Dr. Mohammad (Mo) Pourhomayoun 
#
#  
# 
# File Locations : Crime Data https://www.dropbox.com/s/rhnsu6p95clsojk/Crime_Data_2010_2017.csv?dl=0
#                  MO_Codes   https://www.dropbox.com/s/e3wuydmcgwto5ox/MO_Codes.csv?dl=0
# Instructions: Files should be in the same directory of this file

Import Packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import datetime as dt
from pylab import rcParams
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [70]:
filename = 'Crime_Data_2010_2017.csv'
crime_data = pd.DataFrame()
crime_data = pd.read_csv(filename,nrows=50000)

In [71]:
crime_data.head()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
0,1208575,03/14/2013,03/11/2013,1800,12,77th Street,1241,626,INTIMATE PARTNER - SIMPLE ASSAULT,0416 0446 1243 2000,...,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AO,Adult Other,626.0,NaN,NaN,NaN,6300 BRYNHURST AV,NaN,"(33.9829, -118.3338)"
1,102005556,01/25/2010,01/22/2010,2300,20,Olympic,2071,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,VAN NESS,15TH,"(34.0454, -118.3157)"
2,418,03/19/2013,03/18/2013,2030,18,Southeast,1823,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,200 E 104TH ST,NaN,"(33.942, -118.2717)"
3,101822289,11/11/2010,11/10/2010,1800,18,Southeast,1803,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,88TH,WALL,"(33.9572, -118.2717)"
4,42104479,01/11/2014,01/04/2014,2300,21,Topanga,2133,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329,...,NaN,IC,Invest Cont,745.0,NaN,NaN,NaN,7200 CIRRUS WY,NaN,"(34.2009, -118.6369)"


In [72]:
crime_data.shape

(50000, 26)

In [73]:
list(crime_data)

['DR Number',
 'Date Reported',
 'Date Occurred',
 'Time Occurred',
 'Area ID',
 'Area Name',
 'Reporting District',
 'Crime Code',
 'Crime Code Description',
 'MO Codes',
 'Victim Age',
 'Victim Sex',
 'Victim Descent',
 'Premise Code',
 'Premise Description',
 'Weapon Used Code',
 'Weapon Description',
 'Status Code',
 'Status Description',
 'Crime Code 1',
 'Crime Code 2',
 'Crime Code 3',
 'Crime Code 4',
 'Address',
 'Cross Street',
 'Location ']

# Crime types

In [33]:
crime_data['Crime Code Description'].value_counts().head(20)

BATTERY - SIMPLE ASSAULT                                        5845
THEFT PLAIN - PETTY ($950 & UNDER)                              5091
VEHICLE - STOLEN                                                4627
BURGLARY FROM VEHICLE                                           3402
INTIMATE PARTNER - SIMPLE ASSAULT                               2804
BURGLARY                                                        2747
VANDALISM - MISDEAMEANOR ($399 OR UNDER)                        2509
THEFT OF IDENTITY                                               2300
VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS) 0114    2283
ROBBERY                                                         2223
ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT                  2214
THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)                 1938
CRIMINAL THREATS - NO WEAPON DISPLAYED                          1613
THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD0036    1567
VIOLATION OF COURT ORDER          

# Count of Victim Gender

In [34]:
crime_data['Victim Sex'].value_counts()

M    23853
F    20367
X      472
H        4
Name: Victim Sex, dtype: int64

# Count by Vicitim Race

In [35]:
Victims_bg = {
    "A": "Asian",
    "B": "Black",
    "C": "Chinese",
    "D": "Cambodian",
    "F": "Filipino",
    "G": "Guamanian",
    "H": "Hispanic/Latin/Mexican",
    "I": "American Indian/Alaskan Native",
    "J": "Japanese",
    "K": "Korean",
    "L": "Laotian",
    "O": "Other",
    "P": "Pacific Islander",
    "S": "Samoan",
    "U": "Hawaiian",
    "V": "Vietnamese",
    "W": "White",
    "X": "Unknown",
    "Z": "Asian Indian"
}
crime_data["Victim Descent"] = crime_data["Victim Descent"].map(Victims_bg)
crime_data['Victim Descent'].value_counts()

Hispanic/Latin/Mexican            19097
Black                              9705
White                              9326
Other                              4203
Asian                              1121
Unknown                             955
Korean                              100
Filipino                             60
Chinese                              48
American Indian/Alaskan Native       46
Japanese                             11
Hawaiian                              7
Pacific Islander                      7
Asian Indian                          4
Vietnamese                            3
Guamanian                             2
Laotian                               1
Name: Victim Descent, dtype: int64

# Type of Weapons used

In [36]:
crime_data['Weapon Description'].value_counts().head(20)

STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)    11319
VERBAL THREAT                                      1609
UNKNOWN WEAPON/OTHER WEAPON                         940
HAND GUN                                            704
KNIFE WITH BLADE 6INCHES OR LESS                    381
SEMI-AUTOMATIC PISTOL                               317
OTHER KNIFE                                         290
BOTTLE                                              189
UNKNOWN FIREARM                                     183
REVOLVER                                            140
STICK                                               129
VEHICLE                                             127
BLUNT INSTRUMENT                                    112
FOLDING KNIFE                                       109
KNIFE WITH BLADE OVER 6 INCHES IN LENGTH            108
SIMULATED GUN                                       101
KITCHEN KNIFE                                        99
ROCK/THROWN OBJECT                              

# Crime by periods [Day, Month, Year]

In [74]:
#convert date reported to date object
try:
    date_occ = [dt.datetime.strptime(d, "%m/%d/%Y").date() for d in crime_data["Date Occurred"]]
except:
    print("Error Date Format")
crime_data["Date Occurred"] = np.array(date_occ)

day = [data.isoweekday() for data in crime_data["Date Occurred"]]
mon = [data.month for data in crime_data["Date Occurred"]]
year = [data.year for data in crime_data["Date Occurred"]]

crime_data["Day Occurred"] = np.array(day)
crime_data["Month Occurred"] = np.array(mon)
crime_data["Year Occurred"] = np.array(year)

In [38]:
#Day of The Week
week_day = { 
    1:"Monday",
    2:"Tuesday",
    3:"Wednesday",
    4:"Thursday",
    5:"Friday",
    6:"Saturday",
    7:"Sunday"
}
crime_data['Day Occurred'] = crime_data['Day Occurred'].map(week_day)
crime_data['Day Occurred'].value_counts()

Friday       7842
Saturday     7345
Sunday       7052
Thursday     6991
Monday       6986
Tuesday      6978
Wednesday    6806
Name: Day Occurred, dtype: int64

In [39]:
#Month of The Year
month_year = { 
    1:"January",
    2:"February",
    3:"March",
    4:"April",
    5:"May",
    6:"June",
    7:"July",
    8:"August",
    9:"September",
    10:"October",
    11:"November",
    12:"December"
}
crime_data['Month Occurred'] = crime_data['Month Occurred'].map(month_year)
crime_data['Month Occurred'].value_counts()

August       4678
May          4571
July         4509
March        4488
September    4384
June         4373
January      4347
April        4263
October      3935
February     3800
November     3456
December     3196
Name: Month Occurred, dtype: int64

In [40]:
crime_data['Month Occurred'].describe()[1:]

unique        12
top       August
freq        4678
Name: Month Occurred, dtype: object

In [13]:
#Crime by the Years
crime_data['Year Occurred'].value_counts()

2013    48748
2010     1173
2014       79
Name: Year Occurred, dtype: int64

In [14]:
crime_data['Victim Age'].describe()[1:]

mean    34.609026
std     16.530916
min     10.000000
25%     22.000000
50%     32.000000
75%     46.000000
max     99.000000
Name: Victim Age, dtype: float64

In [15]:
crime_data['Premise Description'].value_counts().head(20)

STREET                                          12219
SINGLE FAMILY DWELLING                           7607
MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)     7462
SIDEWALK                                         3891
PARKING LOT                                      3228
OTHER BUSINESS                                   2574
VEHICLE, PASSENGER/TRUCK                         1673
GARAGE/CARPORT                                    742
DRIVEWAY                                          646
RESTAURANT/FAST FOOD                              636
DEPARTMENT STORE                                  531
HIGH SCHOOL                                       481
HOTEL                                             436
PARK/PLAYGROUND                                   428
OTHER STORE                                       413
MARKET                                            402
ALLEY                                             393
OTHER PREMISE                                     347
NIGHT CLUB (OPEN EVENINGS ON

In [16]:
crime_data['Crime Code'].value_counts().head(20)

624    5845
440    5091
510    4627
330    3402
626    2804
310    2747
745    2509
354    2300
740    2283
210    2223
230    2214
420    1938
930    1613
341    1567
900     629
946     595
649     551
350     524
331     485
888     460
Name: Crime Code, dtype: int64

In [23]:
crime_data['Status Description'].value_counts().head(20)

Invest Cont     36919
Adult Other      6199
Adult Arrest     6192
Juv Arrest        568
Juv Other         121
UNK                 1
Name: Status Description, dtype: int64

In [29]:
crime_data['Status Code'].value_counts().head(20)

IC    36919
AO     6199
AA     6192
JA      568
JO      121
Name: Status Code, dtype: int64

OneHot Encoding function

In [52]:
def one_hot(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode 
    @return a DataFrame with one-hot encoding
    """
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    return df

In [75]:
crime_data['Area Name'].value_counts().head(20)
#list(crime_data)

Southwest      12223
Harbor          8177
Rampart         7888
Central         7373
Hollywood       6575
Hollenbeck      6462
Newton           215
Pacific          190
Mission          131
Olympic          124
77th Street      117
Northeast         99
Topanga           86
Devonshire        77
Southeast         76
Foothill          65
N Hollywood       61
West Valley       57
Van Nuys           3
West LA            1
Name: Area Name, dtype: int64

In [46]:
crime_data['Area ID'].value_counts().head(20)

3     12223
5      8177
2      7888
1      7373
6      6575
4      6462
13      215
14      190
19      131
20      124
12      117
11       99
21       86
17       77
18       76
16       65
15       61
10       57
9         3
8         1
Name: Area ID, dtype: int64

In [107]:
crime_data['Reporting District'].value_counts().head(20)

363    1019
645     680
646     640
636     640
397     534
162     483
111     459
563     451
362     447
392     440
395     437
152     414
358     412
393     401
361     397
237     397
245     383
585     377
666     371
246     370
Name: Reporting District, dtype: int64

In [80]:
crime_data.dtypes

DR Number                   int64
Date Reported              object
Date Occurred              object
Time Occurred               int64
Area ID                     int64
Area Name                  object
Reporting District          int64
Crime Code                  int64
Crime Code Description     object
MO Codes                   object
Victim Age                float64
Victim Sex                 object
Victim Descent             object
Premise Code              float64
Premise Description        object
Weapon Used Code          float64
Weapon Description         object
Status Code                object
Status Description         object
Crime Code 1              float64
Crime Code 2              float64
Crime Code 3              float64
Crime Code 4              float64
Address                    object
Cross Street               object
Location                   object
Day Occurred                int64
Month Occurred              int64
Year Occurred               int64
dtype: object

In [108]:
crime_data.dtypes

DR Number                   int64
Date Reported              object
Date Occurred              object
Time Occurred               int64
Area ID                     int64
Area Name                  object
Reporting District          int64
Crime Code                  int64
Crime Code Description     object
MO Codes                   object
Victim Age                float64
Victim Sex                 object
Victim Descent             object
Premise Code              float64
Premise Description        object
Weapon Used Code          float64
Weapon Description         object
Status Code                object
Status Description         object
Crime Code 1              float64
Crime Code 2              float64
Crime Code 3              float64
Crime Code 4              float64
Address                    object
Cross Street               object
Location                   object
Day Occurred                int64
Month Occurred              int64
Year Occurred               int64
dtype: object

In [123]:
feature_cols = ['Area ID',
                'Day Occurred',
                'Month Occurred',
                'Year Occurred',
                'Time Occurred',
                'Reporting District',
                'Victim Age',
                'Victim Sex',
                'Victim Descent',
                'Premise Code',
                'Weapon Used Code',
                'Status Code']
X = crime_data[feature_cols]
X = one_hot(X,feature_cols)
X = X.drop(feature_cols,axis=1)
X.shape

(50000, 2039)

In [124]:
X.dtypes

Area ID_1                 uint8
Area ID_2                 uint8
Area ID_3                 uint8
Area ID_4                 uint8
Area ID_5                 uint8
Area ID_6                 uint8
Area ID_8                 uint8
Area ID_9                 uint8
Area ID_10                uint8
Area ID_11                uint8
Area ID_12                uint8
Area ID_13                uint8
Area ID_14                uint8
Area ID_15                uint8
Area ID_16                uint8
Area ID_17                uint8
Area ID_18                uint8
Area ID_19                uint8
Area ID_20                uint8
Area ID_21                uint8
Day Occurred_1            uint8
Day Occurred_2            uint8
Day Occurred_3            uint8
Day Occurred_4            uint8
Day Occurred_5            uint8
Day Occurred_6            uint8
Day Occurred_7            uint8
Month Occurred_1          uint8
Month Occurred_2          uint8
Month Occurred_3          uint8
                          ...  
Weapon U

In [125]:
y = crime_data['Crime Code']

In [126]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

In [139]:
knn = KNeighborsClassifier(n_neighbors=501) 

In [140]:
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=501, p=2,
           weights='uniform')

In [141]:
predict_y = knn.predict(X_test)
score = accuracy_score(y_test, predict_y)

In [142]:
print('Accracy score {0:.2f}'.format(score*100))

Accracy score 37.80
